<a href="https://colab.research.google.com/github/royen2000/yolov8/blob/main/yolov8_image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

yolov8 lib install

In [36]:
!pip install torch
!pip install torchvision
!pip install opencv-python
!pip install ultralytics

yolov8 lib install

In [37]:
import torch
from ultralytics import YOLO
import cv2
from matplotlib import pyplot as plt

# YOLOv8 모델 로드
model = YOLO("yolov8s.pt")

# 이미지 로드 및 전처리
image_path = "/content/sample_data/mnist_train_small.csv"  # 예시 이미지 경로
img = cv2.imread(image_path)

if img is None:
    raise FileNotFoundError(f"이미지를 찾을 수 없습니다: {image_path}")

img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# 객체 인식 수행
results = model(img_rgb)

# 결과 검사
if len(results) == 0 or not results[0].boxes:
    raise ValueError("모델 결과가 없습니다. 입력 이미지를 확인하세요.")

# 필터링할 클래스: 신호등(9), 사람(0), 트럭(7), 오토바이(1) - COCO 데이터셋 클래스 인덱스
target_classes = [0, 1, 7, 9]

# 바운딩 박스 데이터로 시각화
for result in results[0].boxes:
    # 바운딩 박스 좌표(x1, y1, x2, y2), 신뢰도(conf), 클래스(cls) 정보 얻기
    x1, y1, x2, y2 = result.xyxy[0].tolist()  # 바운딩 박스 좌표
    conf = result.conf[0].item()  # 신뢰도
    cls = int(result.cls[0].item())  # 클래스 인덱스

    # YOLO 모델의 클래스 이름에 접근하는 방법 수정
    if cls < len(model.names):  # 유효한 클래스 인덱스 범위 체크
        print(f"클래스 인덱스: {cls}, 클래스 이름: {model.names[cls]}, 신뢰도: {conf:.2f}")

        # 라벨 생성 (클래스 이름과 신뢰도)
        label = f"{model.names[cls]} {conf:.2f}"

        # 바운딩 박스 그리기
        cv2.rectangle(img, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
        cv2.putText(img, label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    else:
        print(f"유효하지 않은 클래스 인덱스: {cls}")

# Matplotlib로 결과 시각화
plt.figure(figsize=(10, 10))
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.axis("off")
plt.title("YOLO Detection - Filtered Classes")
plt.show()


FileNotFoundError: 이미지를 찾을 수 없습니다: /content/sample_data/mnist_train_small.csv